### Imports

In [2]:
import os
from time import time
import warnings

In [16]:
from tqdm.autonotebook import tqdm

from statsforecast.models import (
    HistoricAverage,
    Naive,
    RandomWalkWithDrift,
    SeasonalNaive,
    WindowAverage,
    SeasonalWindowAverage,
    SimpleExponentialSmoothing,
    Holt,
    ADIDA,
    CrostonClassic,
    CrostonSBA,
    IMAPA,
    TSB,
    MSTL,
    Theta,
    ARCH,
    ARIMA,
    AutoRegressive,
    AutoARIMA,
)

In [3]:
import pandas as pd
from statsforecast import StatsForecast
from fugue import transform
from datasetsforecast.losses import mse, mae, rmse, mape, smape

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [18]:
warnings.filterwarnings('ignore')

### Constants

In [5]:
DATA_FOLDER = "data/"
M5_FILE = "m5-dataset.parquet.gzip"
FOZZY_FILE = "fozzy-dataset.parquet.gzip"

HORIZON = 14
N_WINDOWS = 1

In [6]:
df = pd.read_parquet(os.path.join(DATA_FOLDER, FOZZY_FILE))
df.y.describe().round(2)

count    26748942.00
mean            0.21
std             1.23
min             0.00
25%             0.00
50%             0.00
75%             0.00
max           801.00
Name: y, dtype: float64

### Code base for Experiments

In [20]:
def convert_classes_to_string(class_list):
    class_string = '-'.join([obj.__class__.__name__ for obj in class_list])
    return class_string


def check_folders():
    folders = [
        FULL_RESULTS_PATH,
        EVALUATION_RESULTS_PATH, 
        METRICS_RESULTS_PATH
    ]
    for path in folders:
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)

In [21]:
class ClassicModel:
    def __init__(self, df: pd.DataFrame, models: list) -> None:
        self.df = df
        self.models = models
        self.metrics = [mse, mae, rmse, mape, smape]
    
    def modeling(self) -> pd.DataFrame:
        sf = StatsForecast(
            models=self.models, 
            freq='D', 
            n_jobs=-1,
        )
        result = sf.cross_validation(
            df=self.df, 
            h=HORIZON, 
            n_windows=N_WINDOWS, 
            step_size=HORIZON, 
            level=[90]
        )
        return result
    
    def evaludate_cross_validation(self, cv_results: pd.DataFrame) -> pd.DataFrame:
        cv_results["unique_id"] = cv_results.index
        cv_results = cv_results.reset_index(drop=True)

        str_models = cv_results.loc[:, ~cv_results.columns.str.contains('unique_id|y|ds|cutoff|lo|hi')].columns
        str_models = ','.join([f"{model}:float" for model in str_models])
        
        evaluation_df = transform(
            cv_results.loc[:, ~cv_results.columns.str.contains('lo|hi')], 
            self.evaluate, 
            params={'metrics': self.metrics}, 
            schema=f"unique_id:str,cutoff:str,metric:str, {str_models}", 
            as_local=True,
            partition={'by': ['unique_id', 'cutoff']}
        )
        return evaluation_df

    def run(self):
        file_name = convert_classes_to_string(self.models) + ".csv"
        check_folders()

        print("Starting cross validation...")
        init = time()
        result = self.modeling()
        end = time()
        print(f"Cross Validation Finished In: {(end - init) / 60}")

        print(f"Saving CV results as {file_name}...")
        results_path = os.path.join(FULL_RESULTS_PATH, file_name)
        result.to_csv(results_path, index=False)

        print("Starting Evaluation...")
        init = time()
        evaluation = self.evaludate_cross_validation(result)
        end = time()
        print(f"Evaluation Finished In: {(end - init) / 60}")

        print(f"Saving evaluation as {file_name}...")
        evaluation.to_csv(os.path.join(EVALUATION_RESULTS_PATH, file_name), index=False)

        print("Saving metrics...")
        metrics = evaluation.groupby(['metric']).mean(numeric_only=True)
        metrics.to_csv(os.path.join(METRICS_RESULTS_PATH, file_name), index=False)
        print(metrics)

        print("Done!")

    @staticmethod
    def evaluate(df: pd.DataFrame, metrics: list) -> pd.DataFrame:
        eval_ = {}
        models = df.loc[:, ~df.columns.str.contains('unique_id|y|ds|cutoff|lo|hi')].columns
        for model in models:
            eval_[model] = {}
            for metric in metrics:
                eval_[model][metric.__name__] = metric(df['y'], df[model])
        eval_df = pd.DataFrame(eval_).rename_axis('metric').reset_index()
        eval_df.insert(0, 'cutoff', df['cutoff'].iloc[0])
        eval_df.insert(0, 'unique_id', df['unique_id'].iloc[0])
        return eval_df

# Run Experiments - Fozzy Group

In [22]:
FULL_RESULTS_PATH = "results/fozzy-full_cv/"
EVALUATION_RESULTS_PATH = "results/fozzy-evaluation/"
METRICS_RESULTS_PATH = "results/fozzy-metrics/"


experiments_config = [

    # HistoricAverage
    {
        "experiment_name": "fozzy-HistoricAverage",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [HistoricAverage()],
    },
    
    # Naive
    {
        "experiment_name": "fozzy-baseline",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [Naive()],
    },
    
    # RandomWalkWithDrift
    {
        "experiment_name": "fozzy-baseline",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [RandomWalkWithDrift()],
    },
    
    # SeasonalNaive
    {
        "experiment_name": "fozzy-baseline",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [SeasonalNaive(season_length=HORIZON)],
    },
    
    # WindowAverage
    {
        "experiment_name": "fozzy-baseline",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [WindowAverage(window_size=HORIZON)],
    },
    
    # SeasonalWindowAverage
    {
        "experiment_name": "fozzy-baseline",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [SeasonalWindowAverage(season_length=HORIZON, window_size=HORIZON)],
    },

    # SimpleExponentialSmoothing
    {
        "experiment_name": "fozzy-exponential-smoothing",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [SimpleExponentialSmoothing(alpha=0.3)],
    },
    
    # Holt
    {
        "experiment_name": "fozzy-exponential-smoothing",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [Holt()],
    },

    # ADIDA
    {
        "experiment_name": "fozzy-sparse-intermittent",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [ADIDA()],
    },
    
    # CrostonClassic
    {
        "experiment_name": "fozzy-sparse-intermittent",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [CrostonClassic()],
    },
    
    # CrostonSBA
    {
        "experiment_name": "fozzy-sparse-intermittent",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [CrostonSBA()],
    },
    
    # IMAPA
    {
        "experiment_name": "fozzy-sparse-intermittent",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [IMAPA()],
    },
    
    # TSB
    {
        "experiment_name": "fozzy-sparse-intermittent",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [TSB(alpha_d=0.2, alpha_p=0.2)],
    },

    # MSTL
    {
        "experiment_name": "fozzy-other",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [MSTL(season_length=HORIZON)],
    },
    
    # Theta
    {
        "experiment_name": "fozzy-other",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [Theta()],
    },
    
    # ARCH
    {
        "experiment_name": "fozzy-other",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [ARCH()],
    },

    # ARIMA
    {
        "experiment_name": "fozzy-ar-arima",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [ARIMA()],
    },
    
    # AutoRegressive
    {
        "experiment_name": "fozzy-ar-arima",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [AutoRegressive(lags=HORIZON)],
    },

    # AutoARIMA
    {
        "experiment_name": "fozzy-AutoARIMA",
        "data": os.path.join(DATA_FOLDER, FOZZY_FILE),
        "models": [AutoARIMA()],
    },
]

In [26]:
experiments_config[18]

{'experiment_name': 'fozzy-AutoARIMA',
 'data': 'data/fozzy-dataset.parquet.gzip',
 'models': [AutoARIMA]}

In [27]:
for config in [experiments_config[18]]:
    print(config, "\n")
    
    df = pd.read_parquet(config["data"])

    try:
        inst = ClassicModel(df, config["models"])
        inst.run()
    except Exception as e:
        print(f"Error with {config['experiment_name']}")
        print(e)

{'experiment_name': 'fozzy-AutoARIMA', 'data': 'data/fozzy-dataset.parquet.gzip', 'models': [AutoARIMA]} 

Starting cross validation...


/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

...

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero en

Error with fozzy-AutoARIMA
division by zero


/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
/Users/oleksandr/opt/anaconda3/envs/phd/lib/python3.10/site-packages/statsforecast/arima.py:896: RuntimeWarning: divide by zero encountered in log
  return 0.5 * np.log(res)
